In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

In [14]:
# get path from data folder
base_path = '../data/enchantments2022.csv'
raw = pd.read_csv(base_path)
raw

,Preferred Entry Date 1,Preferred Division 1,Minimum Acceptable Group Size 1,Maximum Requested Group Size 1,Preferred Entry Date 2,Preferred Division 2,Minimum Acceptable Group Size 2,Maximum Requested Group Size 2,Preferred Entry Date 3,Preferred Division 3,Minimum Acceptable Group Size 3,Maximum Requested Group Size 3,Results Status,Awarded Preference,Awarded Entry Date,Awarded Entrance Code/Name,Awarded Group Size
0,9/2/2022,Core Enchantment Zone,8,8,8/26/2022,Colchuck Zone,8.0,8.0,9/16/2022,Core Enchantment Zone,8.0,8.0,Unsuccessful,NaN,NaN,NaN,NaN
1,8/15/2022,Colchuck Zone,2,2,8/24/2022,Colchuck Zone,2.0,2.0,8/29/2022,Colchuck Zone,2.0,2.0,Unsuccessful,NaN,NaN,NaN,NaN
2,8/12/2022,Snow Zone,8,8,8/19/2022,Snow Zone,8.0,8.0,8/3/2022,Snow Zone,8.0,8.0,Unsuccessful,NaN,NaN,NaN,NaN
3,7/12/2022,Core Enchantment Zone,2,2,7/20/2022,Core Enchantment Zone,2.0,2.0,7/13/2022,Snow Zone,2.0,2.0,Unsuccessful,NaN,NaN,NaN,NaN
4,9/3/2022,Stuart Zone,4,4,8/28/2022,Stuart Zone,4.0,4.0,8/21/2022,Stuart Zone,4.0,4.0,Unsuccessful,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36822,6/28/2022,Core Enchantment Zone,2,2,8/15/2022,Core Enchantment Zone,3.0,3.0,8/22/2022,Core Enchantment Zone,4.0,4.0,Unsuccessful,NaN,NaN,NaN,NaN
36823,8/1/2022,Colchuck Zone,8,8,7/27/2022,Core Enchantment Zone,8.0,8.0,8/6/2022,Core Enchantment Zone,8.0,8.0,Unsuccessful,NaN,NaN,NaN,NaN
36824,7/11/2022,Core Enchantment Zone,4,4,8/22/2022,Core Enchantment Zone,4.0,4.0,8/2/2022,Core Enchantment Zone,4.0,4.0,Unsuccessful,NaN,NaN,NaN,NaN
36825,6/24/2022,Core Enchantment Zone,6,6,8/26/2022,Core Enchantment Zone,8.0,8.0,9/2/2022,Core Enchantment Zone,8.0,8.0,Unsuccessful,NaN,NaN,NaN,NaN


In [15]:
df = raw[raw['Results Status'] != 'Applied']
df = df[df['Results Status'] != 'Cancelled']
df

,Preferred Entry Date 1,Preferred Division 1,Minimum Acceptable Group Size 1,Maximum Requested Group Size 1,Preferred Entry Date 2,Preferred Division 2,Minimum Acceptable Group Size 2,Maximum Requested Group Size 2,Preferred Entry Date 3,Preferred Division 3,Minimum Acceptable Group Size 3,Maximum Requested Group Size 3,Results Status,Awarded Preference,Awarded Entry Date,Awarded Entrance Code/Name,Awarded Group Size
0,9/2/2022,Core Enchantment Zone,8,8,8/26/2022,Colchuck Zone,8.0,8.0,9/16/2022,Core Enchantment Zone,8.0,8.0,Unsuccessful,NaN,NaN,NaN,NaN
1,8/15/2022,Colchuck Zone,2,2,8/24/2022,Colchuck Zone,2.0,2.0,8/29/2022,Colchuck Zone,2.0,2.0,Unsuccessful,NaN,NaN,NaN,NaN
2,8/12/2022,Snow Zone,8,8,8/19/2022,Snow Zone,8.0,8.0,8/3/2022,Snow Zone,8.0,8.0,Unsuccessful,NaN,NaN,NaN,NaN
3,7/12/2022,Core Enchantment Zone,2,2,7/20/2022,Core Enchantment Zone,2.0,2.0,7/13/2022,Snow Zone,2.0,2.0,Unsuccessful,NaN,NaN,NaN,NaN
4,9/3/2022,Stuart Zone,4,4,8/28/2022,Stuart Zone,4.0,4.0,8/21/2022,Stuart Zone,4.0,4.0,Unsuccessful,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36822,6/28/2022,Core Enchantment Zone,2,2,8/15/2022,Core Enchantment Zone,3.0,3.0,8/22/2022,Core Enchantment Zone,4.0,4.0,Unsuccessful,NaN,NaN,NaN,NaN
36823,8/1/2022,Colchuck Zone,8,8,7/27/2022,Core Enchantment Zone,8.0,8.0,8/6/2022,Core Enchantment Zone,8.0,8.0,Unsuccessful,NaN,NaN,NaN,NaN
36824,7/11/2022,Core Enchantment Zone,4,4,8/22/2022,Core Enchantment Zone,4.0,4.0,8/2/2022,Core Enchantment Zone,4.0,4.0,Unsuccessful,NaN,NaN,NaN,NaN
36825,6/24/2022,Core Enchantment Zone,6,6,8/26/2022,Core Enchantment Zone,8.0,8.0,9/2/2022,Core Enchantment Zone,8.0,8.0,Unsuccessful,NaN,NaN,NaN,NaN


In [ ]:
# unique values for prefer